In [17]:
import cv2
import time
import imutils 
from collections import deque

import tensorflow as tf
import os
import shutil
import numpy as np
import random
from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from pathlib import Path
import re
import cv2
from keras import backend as K
tf.__version__
import glob
import imageio

#check if tensorflow gpu is being used
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("tensorflow version:", tf.__version__)

Num GPUs Available:  1
tensorflow version: 2.10.0


V1 (BASIC PROGRAM USING LUCAS KANADE METHOD)

In [5]:
#Link to tutorial: https://www.youtube.com/watch?v=7soIa95QNDk&ab_channel=Pysource 
#Link to website related to tutorial: https://pysource.com/2018/05/14/optical-flow-with-lucas-kanade-method-opencv-3-4-with-python-3-tutorial-31/ 

NeedleViz_path1 = 'Data/edited data/102622_Water.mp4' #long needle
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4' #short needle

#Video Input Choice
cap = cv2.VideoCapture(NeedleViz_path1)
# cap = cv2.VideoCapture(0) #camera

# Mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", select_point)
point_selected = False
point = ()
old_points = np.array([[]])

#Creating old frame
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Lucas Kanade params
lk_params = dict(winSize = (15, 15),
                 maxLevel = 4,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

while True:

    rval, frame = cap.read()

    if rval == True:
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if point_selected is True:
            cv2.circle(frame, point, 5, (0,0,255), 2)

            new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, old_points, None, **lk_params)
            old_gray = gray_frame.copy()
            # print(new_points)
            old_points = new_points

            x, y = new_points.ravel()
            # print(x)
            # print(y)
            cv2.circle(frame, (int(x), int(y)), 5, (0,255,0), -1)

            
        cv2.imshow("Frame", frame)
    
        # Press Q on keyboard to  exit
        if cv2.waitKey(35) & 0xFF == ord('q'): #original waitkey is 25
            break

    #Break out of loop if video is done
    else:
        break 


cap.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

V2 (Implementing Initial Filters) -- KINDA WORKS

In [12]:
#Link to tutorial: https://www.youtube.com/watch?v=7soIa95QNDk&ab_channel=Pysource 
#Link to website related to tutorial: https://pysource.com/2018/05/14/optical-flow-with-lucas-kanade-method-opencv-3-4-with-python-3-tutorial-31/ 

NeedleViz_path1 = 'Data/edited data/102622_Water.mp4' #long needle
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4' #short needle

#Video Input Choice
cap = cv2.VideoCapture(NeedleViz_path2)
# cap = cv2.VideoCapture(0) #camera

# Mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)

cv2.namedWindow("Frame")
cv2.namedWindow("mask")
cv2.setMouseCallback("mask", select_point)
point_selected = False
point = ()
old_points = np.array([[]])

#Lucas Kanade params
lk_params = dict(winSize = (15, 15),
                 maxLevel = 4,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

fgbg = cv2.createBackgroundSubtractorMOG2() #pretty good: (100,200)

frameWidth = 440
frameHeight = 440
cap.set(3, frameWidth)
cap.set(4, frameHeight)

#First Frame Creation
#########################################################
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Initial Frame preprocessing
resized_frame = cv2.resize(old_gray, (frameWidth,frameHeight))

#Achieving desired region of interest within Raw Frame
ROI_frame = resized_frame[94:348, 166:275]
# ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
blank_img = np.zeros_like(resized_frame)
imgContour = blank_img.copy()

x = 94 #initial column number
y = 166 #initial row number

for i in range(0, 254):
    for j in range(0, 109):
        if ROI_frame[i][j] != 0:
            blank_img[x + i, y + j] = ROI_frame[i, j] 

old_fgmask = fgbg.apply(blank_img)

########################################################################

while True:

    rval, frame = cap.read()

    if rval == True:
        
        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

        #Initial Frame preprocessing
        resized_frame = cv2.resize(gray_frame, (frameWidth,frameHeight))

        #Achieving desired region of interest within Raw Frame
        ROI_frame = resized_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = np.zeros_like(resized_frame)
        imgContour = blank_img.copy()

        x = 94 #initial column number
        y = 166 #initial row number
        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 

        fgmask = fgbg.apply(blank_img)

        if point_selected is True:
            cv2.circle(resized_frame, point, 5, (0,0,255), 2)
            cv2.circle(fgmask, point, 5, (0,0,255), 2)

            new_points, status, error = cv2.calcOpticalFlowPyrLK(old_fgmask, fgmask, old_points, None, **lk_params)
            old_fgmask = fgmask.copy()
            # print(new_points)
            old_points = new_points

            x, y = new_points.ravel()
            print('coordinates: X = {}, Y = {}'.format(x,y))
            cv2.circle(resized_frame, (int(x), int(y)), 5, (0,255,0), -1)
            cv2.circle(fgmask, (int(x), int(y)), 5, (0,255,0), -1)


            
        cv2.imshow("Frame", resized_frame)
        cv2.imshow("mask", fgmask)
    
        # Press Q on keyboard to  exit
        if cv2.waitKey(35) & 0xFF == ord('q'): #original waitkey is 25
            break

    #Break out of loop if video is done
    else:
        break 


cap.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.5037841796875
coordinates: X = 226.11753845214844, Y = 613.50378417968

V3 (Implementing More Filters + Contour Detection) --> NOT STABLE 

In [15]:
def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        #Debugging statements
        # if area > 1:
        #     print("Area of contour is: {}".format(area))
        
        areaMin = 60
        areaMax = 250
        if area > areaMin and area < areaMax:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            # print(len(approx))
            x , y , w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5)

            #compute center of contour
            # M = cv2.moments(cnt)
        
            # if M["m00"] != 0:
            #     cX = int(M["m10"] / M["m00"])
            #     cY = int(M["m01"] / M["m00"])
            # else:
            # # set values as what you need in the situation
            #     cX, cY = 0, 0

	        # # draw the contour and center of the shape on the image
            # cv2.circle(imgContour, (cX, cY), 7, (255, 255, 255), -1)


In [18]:
#Link to tutorial: https://www.youtube.com/watch?v=7soIa95QNDk&ab_channel=Pysource 
#Link to website related to tutorial: https://pysource.com/2018/05/14/optical-flow-with-lucas-kanade-method-opencv-3-4-with-python-3-tutorial-31/ 

NeedleViz_path1 = 'Data/edited data/102622_Water.mp4' #long needle
NeedleViz_path2 = 'Data/edited data/102822_Water.mp4' #short needle
NeedleViz_oilAndLatex = 'Data/edited data/oil and latex/capture_5_2022-11-12T16-56-03.mp4'
NeedleViz_gelAndLatex = 'Data/edited data/ultrasound gel and latex/capture_4_2022-11-12T17-33-19.mp4'

#Video Input Choice
cap = cv2.VideoCapture(NeedleViz_path2)
# cap = cv2.VideoCapture(0) #camera

# Mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x, y]], dtype=np.float32)

cv2.namedWindow("Frame")
cv2.namedWindow("mask")
cv2.setMouseCallback("Frame", select_point)
point_selected = False
point = ()
old_points = np.array([[]])

#Lucas Kanade params
lk_params = dict(winSize = (15, 15),
                 maxLevel = 4,
                 criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

fgbg = cv2.createBackgroundSubtractorMOG2() #pretty good: (100,200)

frameWidth = 440
frameHeight = 440
cap.set(3, frameWidth)
cap.set(4, frameHeight)

#First Frame Creation
#########################################################
_, frame = cap.read()
old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#Initial Frame preprocessing
resized_frame = cv2.resize(old_gray, (frameWidth,frameHeight))

#Achieving desired region of interest within Raw Frame
ROI_frame = resized_frame[94:348, 166:275]
# ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
blank_img = np.zeros_like(resized_frame)
imgContour = blank_img.copy()

x = 94 #initial column number
y = 166 #initial row number

for i in range(0, 254):
    for j in range(0, 109):
        if ROI_frame[i][j] != 0:
            blank_img[x + i, y + j] = ROI_frame[i, j] 

old_fgmask = fgbg.apply(blank_img)

temp1 = cv2.GaussianBlur(old_fgmask,(7,7), 1)

temp2 = fgbg.apply(temp1)

getContours(temp2,old_fgmask)

########################################################################

#Preparing to create output videos
image_lst = []

while True:

    rval, frame = cap.read()

    if rval == True:
        
        resized_frame = cv2.resize(frame, (frameWidth,frameHeight))
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

        #Initial Frame preprocessing
        resized_frame = cv2.resize(gray_frame, (frameWidth,frameHeight))

        #Achieving desired region of interest within Raw Frame
        ROI_frame = resized_frame[94:348, 166:275]
         # ROI_frame = resized_frame[col_initial:col_final,row_initial:row_final]
        blank_img = np.zeros_like(resized_frame)
        imgContour = blank_img.copy()

        x = 94 #initial column number
        y = 166 #initial row number
        for i in range(0, 254):
          for j in range(0, 109):

            if ROI_frame[i][j] != 0:
              blank_img[x + i, y + j] = ROI_frame[i, j] 

        fgmask = fgbg.apply(blank_img)

        gaussian = cv2.GaussianBlur(fgmask,(7,7), 1)

        fgmaskV2 = fgbg.apply(gaussian)

        getContours(fgmaskV2,imgContour)


        if point_selected is True:
            cv2.circle(resized_frame, point, 5, (0,0,255), 2)
            cv2.circle(fgmask, point, 5, (0,0,255), 2)

            new_points, status, error = cv2.calcOpticalFlowPyrLK(old_fgmask, imgContour, old_points, None, **lk_params)
            old_fgmask = imgContour.copy()
            old_points = new_points

            x, y = new_points.ravel()
            print('coordinates: X = {}, Y = {}'.format(x,y))
            cv2.circle(resized_frame, (int(x), int(y)), 5, (0,255,0), -1)
            cv2.circle(imgContour, (int(x), int(y)), 5, (0,255,0), -1)


            
        cv2.imshow("Frame", resized_frame)
        cv2.imshow("mask", imgContour)

        #Saving comparison frames as gif 
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)
        overlay = cv2.cvtColor(imgContour, cv2.COLOR_GRAY2BGR)
        stack = np.hstack((resized_frame, overlay))
        # cv2.imshow("stacked", stack)
        image_lst.append(stack)
    
        # Press Q on keyboard to  exit
        if cv2.waitKey(35) & 0xFF == ord('q'): #original waitkey is 25
            break

    #Break out of loop if video is done
    else:
        break 


cap.release() #Release the video capture object

# Close window
cv2.destroyAllWindows()

coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.79124450683594, Y = 582.60986328125
coordinates: X = 222.7912445068

In [20]:
imageio.mimsave('Outputs/V2_velocityTracking_video_10fps.gif', image_lst, fps=10)